In [19]:
# !pip install tensorboard
# !pip install pytorch-lightning

In [20]:
import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim

from pygcn.pygcn.utils import load_data, accuracy
from pygcn.pygcn.models import GCN

from torch.utils.tensorboard import SummaryWriter

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

In [ ]:
writer = SummaryWriter()

In [3]:
seed = 42

np.random.seed(seed)
torch.manual_seed(seed)
torch.mps.manual_seed(seed)

In [4]:
adj, features, labels, idx_train, idx_val, idx_test = load_data('pygcn/data/cora/')

Loading cora dataset...


/Users/i.betev/Desktop/SheafNN/jupiter/pygcn/pygcn/utils.py:80: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:607.)
  return torch.sparse.FloatTensor(indices, values, shape)


In [5]:
hidden = 16
dropout = 0.5
lr = 0.01
weight_decay = 5e-4
device = "cpu"

model = GCN(nfeat=features.shape[1],
            nhid=hidden,
            nclass=labels.max().item() + 1,
            dropout=dropout)

optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

model.to(device)
features = features.to(device)
adj = adj.to(device)
labels = labels.to(device)
idx_train = idx_train.to(device)
idx_val = idx_val.to(device)
idx_test = idx_test.to(device)


In [12]:
def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    # log
    writer.add_scalar("Loss/train", loss_train, epoch)
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    # if not args.fastmode:
    #     # Evaluate validation set performance separately,
    #     # deactivates dropout during validation run.
    #     model.eval()
    #     output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))
    
    
    
    

def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))


In [13]:
epochs = 200

t_total = time.time()
for epoch in range(epochs):
    train(epoch)
    
writer.flush()


print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

Epoch: 0001 loss_train: 1.9285 acc_train: 0.1714 loss_val: 1.9406 acc_val: 0.1600 time: 0.0154s
Epoch: 0002 loss_train: 1.9160 acc_train: 0.2071 loss_val: 1.9326 acc_val: 0.1333 time: 0.0084s
Epoch: 0003 loss_train: 1.9091 acc_train: 0.2071 loss_val: 1.9208 acc_val: 0.1500 time: 0.0052s
Epoch: 0004 loss_train: 1.9018 acc_train: 0.2071 loss_val: 1.9024 acc_val: 0.1600 time: 0.0046s
Epoch: 0005 loss_train: 1.8932 acc_train: 0.2000 loss_val: 1.9034 acc_val: 0.1533 time: 0.0046s
Epoch: 0006 loss_train: 1.8658 acc_train: 0.2143 loss_val: 1.8890 acc_val: 0.1633 time: 0.0042s
Epoch: 0007 loss_train: 1.8540 acc_train: 0.2000 loss_val: 1.8661 acc_val: 0.1600 time: 0.0042s
Epoch: 0008 loss_train: 1.8590 acc_train: 0.2143 loss_val: 1.8683 acc_val: 0.1600 time: 0.0040s
Epoch: 0009 loss_train: 1.8496 acc_train: 0.2071 loss_val: 1.8608 acc_val: 0.1600 time: 0.0039s
Epoch: 0010 loss_train: 1.8249 acc_train: 0.2929 loss_val: 1.8431 acc_val: 0.2067 time: 0.0043s
Epoch: 0011 loss_train: 1.8233 acc_train

In [14]:
writer.close()

In [15]:
!tensorboard --logdir=runs

TensorFlow installation not found - running with reduced feature set.
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.15.1 at http://localhost:6006/ (Press CTRL+C to quit)
^C


In [ ]:
# Testing
test()